In [1]:
import pandas as pd
from crypto_momentum_portfolios.data_loader import CryptoDataLoader
from crypto_momentum_portfolios.indicators import Indicators

In [2]:
dl = CryptoDataLoader()

In [3]:
btc_eth = dl.get_crypto(["bitcoin","ethereum"])
btc = dl.get_crypto("bitcoin")

In [4]:
btc_eth

,bitcoin,ethereum
date,,
2021-01-01,29022.418395,738.616938
2021-01-02,29352.126792,730.147340
2021-01-03,32163.824935,777.696065
2021-01-04,33008.226203,967.000597
2021-01-05,31515.575967,1025.654768
...,...,...
2023-06-25,30537.816665,1875.059962
2023-06-26,30454.756974,1899.147312
2023-06-27,30285.505064,1859.802420


In [6]:
Indicators.momentum(btc_eth)

,bitcoin,ethereum
date,,
2021-01-01,1.000000,1.000000
2021-01-02,1.000000,1.000000
2021-01-03,1.000000,1.000000
2021-01-04,1.000000,1.000000
2021-01-05,1.000000,1.000000
...,...,...
2023-06-25,1.138447,1.007162
2023-06-26,1.117698,0.995771
2023-06-27,1.118935,0.982942


In [10]:
Indicators.momentum(btc)

,bitcoin
date,
2021-01-01,1.000000
2021-01-02,1.000000
2021-01-03,1.000000
2021-01-04,1.000000
2021-01-05,1.000000
...,...
2023-06-25,1.138447
2023-06-26,1.117698
2023-06-27,1.118935


In [14]:
btc = btc.asfreq("1D")
btc.index

DatetimeIndex(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04',
               '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08',
               '2021-01-09', '2021-01-10',
               ...
               '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23',
               '2023-06-24', '2023-06-25', '2023-06-26', '2023-06-27',
               '2023-06-28', '2023-06-29'],
              dtype='datetime64[ns]', name='date', length=910, freq='D')